### Import libraries and packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
import torchmetrics

import random
import os

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

from torchvision.datasets import CIFAR10, CIFAR100

#### Seeds

In [ ]:
torch.manual_seed(4)
torch.cuda.manual_seed(4)

random.seed(4)
np.random.seed(4)

#### Cuda

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

### Work with data

#### Upload

In [ ]:
data_dir = './data/'

##### Task 4.1

In [ ]:
X_train = pd.read_csv(os.path.join(data_dir,'xtrain.csv'), header=None) 
Y_train = pd.read_csv(os.path.join(data_dir,'ytrain.csv'), header=None) 
X_test = pd.read_csv(os.path.join(data_dir,'xtest.csv'), header=None) 
Y_test = pd.read_csv(os.path.join(data_dir,'ytest.csv'), header=None) 

In [ ]:
# Run 1 time
# convert to torch.tensor
print("Type before:", type(X_train), type(X_test), type(Y_train), type(Y_test))
X_train = torch.tensor(X_train.values)
X_test = torch.tensor(X_test.values)
Y_train = torch.tensor(Y_train.values)
Y_test = torch.tensor(Y_test.values)
print("Type after:", type(X_train), type(X_test), type(Y_train), type(Y_test))

##### Task 4.2

In [ ]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

raw_dataset = pd.read_csv(url, names=column_names,
                          na_values='?', comment='\t',
                          sep=' ', skipinitialspace=True)

In [ ]:
raw_dataset.dropna(inplace=True)
dataset = raw_dataset.copy()
#dataset['Origin'] = dataset['Origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})
#dataset = pd.get_dummies(dataset, columns=['Origin'], prefix='', prefix_sep='')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset.iloc[:,1:dataset.shape[1]], dataset.iloc[:,0], train_size=0.8, random_state=42, shuffle=True)

In [ ]:
# Run 1 time
# convert to torch.tensor
print("Type before:", type(X_train), type(X_test), type(y_train), type(y_test))
X_train = torch.tensor(X_train.values)
X_test = torch.tensor(X_test.values)
y_train = torch.tensor(y_train.values)
y_test = torch.tensor(y_test.values)
print("Type after:", type(X_train), type(X_test), type(y_train), type(y_test))

##### Task 4.3

This task include 2 models:
+ For CIFAR-10
+ For CIFAR-100

Before training, please download CIFAR-10 or CIFAR-100

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(), 
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.5)])

batch_size = 20

trainset = CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=12)

testset = CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=12)

classes = trainset.classes

#### Visualization

##### Task 4.1

In [ ]:
X_train.hist(figsize=(14,10))

In [ ]:
X_test.hist(figsize=(14,10))

##### Task 4.2

In [ ]:
raw_dataset.hist(figsize=(14,10))

##### Task 4.3

In [ ]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
# get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

### Creating class of models for task 4.1

#### Functions

In [ ]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred))*100
    return acc

#### Model 1

In [ ]:
class ModelNet1(torch.nn.Module):
    """
    This class construct model.
    """
    def __init__(self, input_features):
        super().__init__()
        self.layers_stack = torch.nn.Sequential(
            torch.nn.Linear(in_features=input_features, out_features=16, dtype=float),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=16, out_features=8, dtype=float),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=8, out_features=6, dtype=float),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=6, out_features=1, dtype=float),
            #torch.nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.layers_stack(x)
    

model_net1 = ModelNet1(input_features=30).to(device)

In [ ]:
loss = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_net1.parameters(), lr=1.00e-3)

In [ ]:
test_acc_history = []
test_loss_history = []
train_acc_history = []
train_loss_history = []

X_train, Y_train = X_train.to(device), Y_train.to(device)
X_test, Y_test = X_test.to(device), Y_test.to(device)

for epoch in range(1000):
    optimizer.zero_grad()
    model_net1.train()

    y_logits = model_net1(X_train)
    y_pred = torch.round(torch.sigmoid(y_logits))
    loss = torch.nn.BCEWithLogitsLoss()(y_logits, Y_train)
    acc = accuracy_fn(y_true=Y_train, y_pred=y_pred)
    #acc = (y_pred.argmax() == Y_train).float().mean().data.cpu()
    
    train_acc_history.append(acc)
    train_loss_history.append(loss)

    
    loss.backward()
    optimizer.step()

    model_net1.eval()
    with torch.inference_mode():
        # 1. Forward pass
        test_logits = model_net1(X_test)
        test_pred = torch.round(torch.sigmoid(test_logits))
        # 2. Caculate loss/accuracy
        test_loss = torch.nn.BCEWithLogitsLoss()(test_logits, Y_test)
        test_acc = accuracy_fn(y_true=Y_test, y_pred=test_pred)
        #test_acc = (test_pred.argmax() == Y_test).float().mean().data.cpu()

        test_acc_history.append(test_acc)
        test_loss_history.append(test_loss)
    
    #print(f'Train acc {acc:.5f}, Test acc {test_acc:.5f}')
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Train loss: {loss:.5f}, Train acc: {acc:.5f}% | Test loss: {test_loss:.5f}, Test acc: {test_acc:.5f}%")


#### Model 2

In [ ]:
class ModelNet2(torch.nn.Module):
    """
    This class construct model.
    """
    def __init__(self, input_features):
        super().__init__()
        self.layers_stack = torch.nn.Sequential(
            torch.nn.Linear(in_features=input_features, out_features=20, dtype=float),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=20, out_features=15, dtype=float),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=15, out_features=10, dtype=float),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=10, out_features=5, dtype=float),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=5, out_features=1, dtype=float),
            #torch.nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.layers_stack(x)
    

model_net2 = ModelNet2(input_features=30).to(device)

In [ ]:
loss = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_net2.parameters(), lr=1.00e-3)

In [ ]:
test_acc_history = []
test_loss_history = []
train_acc_history = []
train_loss_history = []

X_train, Y_train = X_train.to(device), Y_train.to(device)
X_test, Y_test = X_test.to(device), Y_test.to(device)

for epoch in range(1000):
    optimizer.zero_grad()
    model_net2.train()

    y_logits = model_net2(X_train)
    y_pred = torch.round(torch.sigmoid(y_logits))
    loss = torch.nn.BCEWithLogitsLoss()(y_logits, Y_train)
    acc = accuracy_fn(y_true=Y_train, y_pred=y_pred)
    #acc = (y_pred.argmax() == Y_train).float().mean().data.cpu()
    
    train_acc_history.append(acc)
    train_loss_history.append(loss)

    
    loss.backward()
    optimizer.step()

    model_net2.eval()
    with torch.inference_mode():
        # 1. Forward pass
        test_logits = model_net2(X_test)
        test_pred = torch.round(torch.sigmoid(test_logits))
        # 2. Caculate loss/accuracy
        test_loss = torch.nn.BCEWithLogitsLoss()(test_logits, Y_test)
        test_acc = accuracy_fn(y_true=Y_test, y_pred=test_pred)
        #test_acc = (test_pred.argmax() == Y_test).float().mean().data.cpu()

        test_acc_history.append(test_acc)
        test_loss_history.append(test_loss)
    
    #print(f'Train acc {acc:.5f}, Test acc {test_acc:.5f}')
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Train loss: {loss:.5f}, Train acc: {acc:.5f}% | Test loss: {test_loss:.5f}, Test acc: {test_acc:.5f}%")


#### Model 3

In [ ]:
class ModelNet3(torch.nn.Module):
    """
    This class construct model.
    """
    def __init__(self, input_features):
        super().__init__()
        self.layers_stack = torch.nn.Sequential(
            torch.nn.Linear(in_features=input_features, out_features=20, dtype=float),
            torch.nn.Tanh(),
            torch.nn.Linear(in_features=20, out_features=15, dtype=float),
            torch.nn.Tanh(),
            torch.nn.Linear(in_features=15, out_features=10, dtype=float),
            torch.nn.Tanh(),
            torch.nn.Linear(in_features=10, out_features=5, dtype=float),
            torch.nn.Tanh(),
            torch.nn.Linear(in_features=5, out_features=1, dtype=float),
            #torch.nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.layers_stack(x)
    

model_net3 = ModelNet3(input_features=30).to(device)

In [ ]:
loss = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model_net3.parameters(), lr=1.00e-3)

In [ ]:
test_acc_history = []
test_loss_history = []
train_acc_history = []
train_loss_history = []

X_train, Y_train = X_train.to(device), Y_train.to(device)
X_test, Y_test = X_test.to(device), Y_test.to(device)

for epoch in range(1000):
    optimizer.zero_grad()
    model_net3.train()

    y_logits = model_net3(X_train)
    y_pred = torch.round(torch.sigmoid(y_logits))
    loss = torch.nn.BCEWithLogitsLoss()(y_logits, Y_train)
    acc = accuracy_fn(y_true=Y_train, y_pred=y_pred)
    #acc = (y_pred.argmax() == Y_train).float().mean().data.cpu()
    
    train_acc_history.append(acc)
    train_loss_history.append(loss)

    
    loss.backward()
    optimizer.step()

    model_net3.eval()
    with torch.inference_mode():
        # 1. Forward pass
        test_logits = model_net3(X_test)
        test_pred = torch.round(torch.sigmoid(test_logits))
        # 2. Caculate loss/accuracy
        test_loss = torch.nn.BCEWithLogitsLoss()(test_logits, Y_test)
        test_acc = accuracy_fn(y_true=Y_test, y_pred=test_pred)
        #test_acc = (test_pred.argmax() == Y_test).float().mean().data.cpu()

        test_acc_history.append(test_acc)
        test_loss_history.append(test_loss)
    
    #print(f'Train acc {acc:.5f}, Test acc {test_acc:.5f}')
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Train loss: {loss:.5f}, Train acc: {acc:.5f}% | Test loss: {test_loss:.5f}, Test acc: {test_acc:.5f}%")


#### Visualization

In [ ]:
plt.plot(train_acc_history)
plt.plot(test_acc_history)
plt.legend(['train_acc', 'test_acc'])
plt.title('accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy value in %')

In [ ]:
plt.plot(test_loss_history)
plt.title('test_loss')
plt.xlabel('epochs')
plt.ylabel('loss value')

### Creating class of model for task 4.2

#### Model

In [ ]:
class ModelRegNet(torch.nn.Module):
    """
    This class construct model.
    """
    def __init__(self, input_features):
        super().__init__()
        self.layers_stack = torch.nn.Sequential(
            torch.nn.Linear(in_features=input_features, out_features=17, dtype=float),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=17, out_features=13, dtype=float),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=13, out_features=10, dtype=float),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=10, out_features=6, dtype=float),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=6, out_features=1, dtype=float)
        )
    
    def forward(self, x):
        return self.layers_stack(x)
    
modelreg_net = ModelRegNet(input_features=dataset.shape[1] - 1).to(device)

In [ ]:
loss = torch.nn.MSELoss()
optimizer = torch.optim.Adam(modelreg_net.parameters(), lr=1.00e-3)

In [ ]:
test_loss_history = []
test_acc_history = []

train_loss_history = []
train_acc_history = []

X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

for epoch in range(5000):
    optimizer.zero_grad()
    modelreg_net.train()

    y_pred = modelreg_net(X_train).squeeze()
    loss_v = loss(y_pred, y_train)

    # Separate part with gradients
    loss_v_copy = loss_v.clone()
    loss_v_copy = loss_v_copy.detach().numpy()
    train_loss_history.append(loss_v_copy)

    y_pred_copy = y_pred.clone().detach().numpy()
    y_train_copy = y_train.clone().detach().numpy()
    train_acc_history.append(r2_score(y_true=y_train_copy, y_pred=y_pred_copy))

    loss_v.backward()
    optimizer.step()

    modelreg_net.eval()
    with torch.inference_mode():
        # 1. Forward pass
        test_pred = modelreg_net(X_test).squeeze()
        # 2. Caculate loss/accuracy
        loss_v_test = loss(test_pred, y_test)

        test_loss_history.append(loss_v_test)
        test_acc_history.append(r2_score(y_true=y_test, y_pred=test_pred))
    
    #print(f'Train acc {acc:.5f}, Test acc {test_acc:.5f}')
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Train loss: {loss_v:.5f} | Test loss: {loss_v_test:.5f} | ")


#### Visualization

In [ ]:
plt.plot(train_loss_history)
plt.plot(test_loss_history)
plt.legend(['train_loss', 'test_loss'])
plt.title('test_loss')
plt.xlabel('epochs')
plt.ylabel('loss value')

In [ ]:
plt.plot(train_acc_history)
plt.plot(test_acc_history)
plt.legend(['train_acc', 'test_acc'])
plt.title('accuracy r2_score')
plt.xlabel('epochs')
plt.ylabel('accuracy value')

### Creating class of model for task 4.3

#### Model

In [ ]:
class ClfC10Net(torch.nn.Module):
   
    def __init__(self):
        
        super(ClfC10Net, self).__init__()

        self.conv_layer = torch.nn.Sequential(

            # Conv Layer block 1
            torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            torch.nn.BatchNorm2d(32),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 2
            torch.nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            torch.nn.BatchNorm2d(128),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Dropout2d(p=0.05),

            # Conv Layer block 3
            torch.nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            torch.nn.BatchNorm2d(256),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=2, stride=2), # 4
        )


        self.fc_layer = torch.nn.Sequential(
            torch.nn.Dropout(p=0.1),
            torch.nn.Linear(4096, 1024),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(1024, 512),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(p=0.1),
            torch.nn.Linear(512, 10)
        )


    def forward(self, x):
        
        # conv layers
        x = self.conv_layer(x)
        
        # flatten
        x = x.view(x.size(0), -1)
        
        # fc layer
        x = self.fc_layer(x)

        return x

clfc10_net = ClfC10Net().to(device)

In [ ]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(clfc10_net.parameters(), lr=1e-4)

In [ ]:
metric_pr = torchmetrics.Precision(task='multiclass', num_classes=10)

In [ ]:
test_loss_history = []
test_acc_history = []

train_loss_history = []
train_acc_history = []

telh_ep = []
teah_ep = []

trlh_ep = []
trah_ep = []

for epoch in range(100):

    running_loss_train = 0
    correct_train = 0
    total_train = 0

    running_loss_test = 0
    correct_test = 0
    total_test = 0

    for i, data in enumerate(trainloader, 0):
        clfc10_net.train()
        inputs_train, labels_train = data
        
        optimizer.zero_grad()

        pred_train = clfc10_net.forward(inputs_train.to(device))
        loss_train = loss(pred_train.to(device), labels_train.to(device))
        
        loss_train_copy = loss_train.clone().cpu().detach().numpy()
        train_loss_history.append(loss_train_copy)

        pred_train_copy = pred_train.clone().cpu().detach()
        labels_train_copy = labels_train.clone().cpu().detach()

        running_loss_train += loss_train_copy.item()

        loss_train.backward()
        optimizer.step()
        _, predicted = pred_train_copy.max(1)

        total_train += labels_train_copy.size(0)
        correct_train += predicted.eq(labels_train_copy).sum().item()
    
    acc_train = 100 * correct_train / total_train
    train_loss=running_loss_train/len(trainloader)

    for i, data in enumerate(testloader, 0):
      clfc10_net.eval()
      inputs_test, labels_test = data
      with torch.no_grad():
        pred_test = clfc10_net.forward(inputs_test.to(device))
        loss_test = loss(pred_test.to(device), labels_test.to(device))

        loss_test_copy = loss_test.clone().cpu().detach().numpy()
        test_loss_history.append(loss_test_copy)

        pred_test_copy = pred_test.clone().cpu().detach()
        labels_test_copy = labels_test.clone().cpu().detach()

        running_loss_test += loss_test_copy.item()

        _, predicted = pred_test_copy.max(1)
        total_test += labels_test_copy.size(0)
        correct_test += predicted.eq(labels_test_copy).sum().item()
        

    acc_test = 100 * correct_test / total_test
    test_loss=running_loss_test/len(testloader)

    telh_ep.append(test_loss)
    trlh_ep.append(train_loss)
    
    trah_ep.append(acc_train)
    teah_ep.append(acc_test)
    print(f'------- epoch {epoch + 1} -------')
    print(f'loss_train: {train_loss}')
    print(f'loss_test: {test_loss}')
    print()
    print(f'acc_train: {acc_train}')
    print(f'acc_test: {acc_test}')

#### Model cifar-100

In [ ]:
class ClfC100Net(torch.nn.Module):
  
    def __init__(self):
        
        super(ClfC100Net, self).__init__()

        self.conv_layer = torch.nn.Sequential(

           torch.nn.Conv2d(3, 64, 3, 1, 1), # 32
           torch.nn.BatchNorm2d(64),
           torch.nn.ReLU(inplace=True),

           torch.nn.Conv2d(64, 128, 3, 1, 1), # 32
           torch.nn.BatchNorm2d(128),
           torch.nn.ReLU(inplace=True),
           torch.nn.MaxPool2d(2, 2, 0), # 16

           torch.nn.Conv2d(128, 128, 3, 1, 1), # 16
           torch.nn.BatchNorm2d(128),
           torch.nn.ReLU(inplace=True),

           torch.nn.Conv2d(128, 128, 3, 1, 1), # 16
           torch.nn.BatchNorm2d(128),
           torch.nn.ReLU(inplace=True),

           torch.nn.Conv2d(128, 256, 3, 1, 1), # 16
           torch.nn.BatchNorm2d(256),
           torch.nn.ReLU(inplace=True),
           torch.nn.MaxPool2d(2, 2, 0), # 8

           torch.nn.Conv2d(256, 512, 3, 1, 1), # 8
           torch.nn.BatchNorm2d(512),
           torch.nn.ReLU(inplace=True),
           torch.nn.MaxPool2d(2, 2, 0), # 4

           torch.nn.Conv2d(512, 512, 3, 1, 1), # 8
           torch.nn.BatchNorm2d(512),
           torch.nn.ReLU(inplace=True),
           torch.nn.MaxPool2d(4, 4, 0) # 4
        )


        self.fc_layer = torch.nn.Sequential(
            torch.nn.Flatten(),
            torch.nn.Dropout(p=0.1, inplace=False),
            torch.nn.Linear(512, 100),
        )


    def forward(self, x):
        """Perform forward."""
        
        # conv layers
        x = self.conv_layer(x)
        
        # flatten
        x = x.view(x.size(0), -1)
        
        # fc layer
        x = self.fc_layer(x)

        return x

clfc100_net = ClfC100Net().to(device)

In [ ]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(clfc100_net.parameters(), lr=1e-3)

In [ ]:
test_loss_history = []
test_acc_history = []

train_loss_history = []
train_acc_history = []

telh_ep = []
teah_ep = []

trlh_ep = []
trah_ep = []

for epoch in range(100):

    running_loss_train = 0
    correct_train = 0
    total_train = 0

    running_loss_test = 0
    correct_test = 0
    total_test = 0

    for i, data in enumerate(trainloader, 0):
        clfc100_net.train()
        inputs_train, labels_train = data
        
        optimizer.zero_grad()

        pred_train = clfc100_net.forward(inputs_train.to(device))
        loss_train = loss(pred_train.to(device), labels_train.to(device))

        loss_train_copy = loss_train.clone().cpu().detach().numpy()
        train_loss_history.append(loss_train_copy)

        pred_train_copy = pred_train.clone().cpu().detach()
        labels_train_copy = labels_train.clone().cpu().detach()

        running_loss_train += loss_train_copy.item()

        loss_train.backward()
        optimizer.step()
        _, predicted = pred_train_copy.max(1)

        total_train += labels_train_copy.size(0)
        correct_train += predicted.eq(labels_train_copy).sum().item()
    
    acc_train = 100 * correct_train / total_train
    train_loss=running_loss_train/len(trainloader)

    for i, data in enumerate(testloader, 0):
      clfc100_net.eval()
      inputs_test, labels_test = data
      with torch.no_grad():
        pred_test = clfc100_net.forward(inputs_test.to(device))
        loss_test = loss(pred_test.to(device), labels_test.to(device))

        loss_test_copy = loss_test.clone().cpu().detach().numpy()
        test_loss_history.append(loss_test_copy)

        pred_test_copy = pred_test.clone().cpu().detach()
        labels_test_copy = labels_test.clone().cpu().detach()

        running_loss_test += loss_test_copy.item()

        _, predicted = pred_test_copy.max(1)
        total_test += labels_test_copy.size(0)
        correct_test += predicted.eq(labels_test_copy).sum().item()


    acc_test = 100 * correct_test / total_test
    test_loss=running_loss_test/len(testloader)

    telh_ep.append(test_loss)
    trlh_ep.append(train_loss)
    
    trah_ep.append(acc_train)
    teah_ep.append(acc_test)
    print(f'------- epoch {epoch + 1} -------')
    print(f'loss_train: {train_loss}')
    print(f'loss_test: {test_loss}')
    print()
    print(f'acc_train: {acc_train}')
    print(f'acc_test: {acc_test}')

#### Visualization

In [ ]:
plt.plot(trlh_ep)
plt.plot(telh_ep)
plt.legend(['train_loss', 'test_loss'])
plt.title('loss')
plt.xlabel('epochs')
plt.ylabel('loss value')

In [ ]:
plt.plot(trah_ep)
plt.plot(teah_ep)
plt.legend(['train_acc', 'test_acc'])
plt.title('accuracy (Precision)')
plt.xlabel('epochs')
plt.ylabel('acc value')